In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import keras
from keras import layers
from keras.datasets import mnist
from keras import backend as K
from ipywidgets import interact, widgets
from IPython.display import display

# Function to load and preprocess RGB images
def load_and_preprocess_images(folder_path, class_label, target_size=(32, 32)):
    data = []
    for image_file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_file)
        img = Image.open(image_path) # Convert to RGB
        img = img.resize(target_size, Image.LANCZOS)  # Use LANCZOS resampling
        img_array = np.array(img) / 255.0
        data.append((img_array.flatten(), class_label))
    return data

# Load and preprocess RGB images from folder 1
folder_1_data = load_and_preprocess_images('/content/drive/MyDrive/MAG', 'Class_1')

# Load and preprocess RGB images from folder 2
folder_2_data = load_and_preprocess_images('/content/drive/MyDrive/ME', 'Class_2')

# Combine the data from both folders
data = folder_1_data + folder_2_data
df = pd.DataFrame(data, columns=['Image_Array', 'Class_Label'])

# Convert the DataFrame to a NumPy array
images = np.array(df['Image_Array'].tolist())
labels = df['Class_Label'].values

print("images",images)
print("images shape",images.shape)

print("labels",labels)

# Get the size of the images
original_dim = images[0].shape[0]
print("original_dim", original_dim)

# One-hot encode class labels
num_classes = len(np.unique(labels))
print("num_classes", num_classes)
labels_onehot = keras.utils.to_categorical(pd.Categorical(labels).codes, num_classes)


images [[0.6627451  0.64313725 0.63137255 ... 0.42745098 0.41960784 0.43529412]
 [0.77254902 0.77254902 0.77254902 ... 0.3372549  0.35686275 0.39607843]
 [0.82352941 0.82352941 0.82352941 ... 0.32941176 0.31372549 0.30980392]
 ...
 [0.61176471 0.63921569 0.64313725 ... 0.09019608 0.08627451 0.0745098 ]
 [0.65490196 0.69019608 0.68235294 ... 0.14117647 0.1372549  0.11764706]
 [0.68235294 0.72156863 0.7254902  ... 0.16078431 0.15294118 0.1372549 ]]
images shape (200, 3072)
labels ['Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1'
 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1'
 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1'
 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1'
 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1'
 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1'
 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1' 'Class_1'
 'Class_1' 'Class_1'

In [3]:
import keras
from keras import layers
import numpy as np
from keras import backend as K
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from IPython.display import display

# Set random seed for reproducibility
np.random.seed(1000)

original_dim = 32 * 32 * 3
intermediate_dim = 256
latent_dim = 64
num_classes = 2

# Input for both encoder and decoder
inputs = keras.Input(shape=(original_dim,))

# Encoder
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

# Define conditional input (class label)
class_label = layers.Input(shape=(num_classes,))

# Concatenate class label with input
concatenated_input = layers.concatenate([h, class_label])

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

# Sample latent space
z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

# Create encoder model
encoder = keras.Model([inputs, class_label], [z_mean, z_log_sigma, z])

# Decoder
latent_inputs = layers.Input(shape=(latent_dim,))
concatenated_inputs = layers.concatenate([latent_inputs, class_label])
x = layers.Dense(intermediate_dim, activation='relu')(concatenated_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)

# Create decoder model
decoder = keras.Model([latent_inputs, class_label], outputs)

# Instantiate VAE model
outputs = decoder([encoder([inputs, class_label])[2], class_label])
vae = keras.Model([inputs, class_label], outputs)

# Reconstruction loss
reconstruction_loss = keras.losses.mean_squared_error(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)


# Train VAE model
vae.compile(optimizer='adam')  # No need to specify loss again, as it's already added in the model
vae.fit([images, labels_onehot], images, epochs=5, batch_size=16)

num_images_slider_vae = widgets.IntSlider(value=1, min=1, max=10, step=1)

@interact(num_images=num_images_slider_vae)
def generate_images_vae(num_images=1):  # Default value set to 1
    if num_images <= 0:
        return

    # Sample random latent vectors
    random_latent_vectors = np.random.normal(size=(num_images, latent_dim))

    # Generate images from the sampled latent vectors
    generated_images = decoder.predict([random_latent_vectors, np.zeros((num_images, num_classes))])

    # Display generated images
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(generated_images[i].reshape(32, 32, 3), cmap='viridis')  # Use viridis colormap for better visibility
        plt.axis('off')
    plt.show()


Epoch 1/5
13/13 [==============================] - 2s 21ms/step - loss: 183.7737
Epoch 2/5
13/13 [==============================] - 0s 21ms/step - loss: 119.8208
Epoch 3/5
13/13 [==============================] - 0s 21ms/step - loss: 109.4618
Epoch 4/5
13/13 [==============================] - 0s 20ms/step - loss: 96.3858
Epoch 5/5
13/13 [==============================] - 0s 21ms/step - loss: 82.1576


interactive(children=(IntSlider(value=1, description='num_images', max=10, min=1), Output()), _dom_classes=('w…

In [4]:
num_images_slider_cvae = widgets.IntSlider(value=1, min=1, max=10, step=1)

@interact(num_images=num_images_slider_cvae)
def generate_images_cvae(num_images=1):  # Default value set to 1
    if num_images <= 0:
        return

    # Select a random class label for each image
    random_class_labels = np.random.randint(0, num_classes, num_images)
    random_class_labels_onehot = keras.utils.to_categorical(random_class_labels, num_classes)

    # Generate images from the sampled latent vectors and class labels
    generated_images = vae.predict([images[:num_images], random_class_labels_onehot])

    # Display generated images
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(generated_images[i].reshape(32, 32, 3), cmap='viridis')  # Use viridis colormap for better visibility
        plt.axis('off')
    plt.show()


interactive(children=(IntSlider(value=1, description='num_images', max=10, min=1), Output()), _dom_classes=('w…